In [1]:
import os
import pandas as pd
from helper import create_folder

In [2]:
def combine_csv_files(folder_path, file_suffix):
    dfs = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(file_suffix):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                match_uid = os.path.basename(os.path.dirname(file_path))
                df['match_uid'] = str(match_uid)
                dfs.append(df)
    
    combined_df = pd.concat(dfs, axis=0, ignore_index=True)
    return combined_df

# Combine _outfield.csv files
outfield_folder_path = "match_data"
outfield_suffix = "_outfield.csv"
outfield_combined_df = combine_csv_files(outfield_folder_path, outfield_suffix)

# Combine _keeper.csv files
keeper_folder_path = "match_data"
keeper_suffix = "_keeper.csv"
keeper_combined_df = combine_csv_files(keeper_folder_path, keeper_suffix)

# Save the combined dataframes to CSV files
create_folder("combined_match_data")
outfield_combined_df.to_csv("combined_match_data/outfield_combined.csv", index=False)
keeper_combined_df.to_csv("combined_match_data/keeper_combined.csv", index=False)

In [3]:
import csv

# Specify the input CSV files
outfield_csv = 'combined_match_data/outfield_combined.csv'
keeper_csv = 'combined_match_data/keeper_combined.csv'

# Specify the output CSV files
outfield_processed = 'combined_match_data/outfield_processed.csv'
keeper_processed = 'combined_match_data/keeper_processed.csv'

# Specify the value to replace empty cells
replacement_value = 'NULL'  # or any other value that MySQL Workbench recognizes

# Function to process the CSV file
def process_csv(input_file, output_file):
    with open(input_file, 'r') as file_in, open(output_file, 'w', newline='') as file_out:
        reader = csv.reader(file_in)
        writer = csv.writer(file_out)

        for row in reader:
            processed_row = [cell if cell else replacement_value for cell in row]
            writer.writerow(processed_row)

# Process the outfield CSV
process_csv(outfield_csv, outfield_processed)

# Process the keeper CSV
process_csv(keeper_csv, keeper_processed)


In [7]:
import re

# Function to convert to snake_case
def convert_to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

# List of CSV files
csv_files = [
    '/Users/limjohn/Documents/GitHub/fpl-scraping/team_data/team_data_epl_22_23.csv',
    '/Users/limjohn/Documents/GitHub/fpl-scraping/match_data/all_match_links_epl_22_23.csv',
    '/Users/limjohn/Documents/GitHub/fpl-scraping/player_data/players_epl_22_23.csv',
    '/Users/limjohn/Documents/GitHub/fpl-scraping/combined_match_data/keeper_processed.csv',
    '/Users/limjohn/Documents/GitHub/fpl-scraping/combined_match_data/outfield_processed.csv'
]

# Process each CSV file
for csv_file in csv_files:
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Convert column names to snake_case
    df.columns = [convert_to_snake_case(col) for col in df.columns]

    # Specific case for 'all_match_links_epl_22_23.csv'
    if 'all_match_links_epl_22_23.csv' in csv_file:
        df.rename(columns={'x_g': 'xg_home', 'x_g.1': 'xg_away'}, inplace=True)

    # Write the data back to the CSV file
    df.to_csv(csv_file, index=False)

print("All CSV files processed.")


All CSV files processed.
